In [15]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from src.tradecli import *
from src.utils import *
from src.journal import  *

INFO:root:Creating initial trades
INFO:root:Adding more trades to the journal
INFO:root:Adding default tags to all trades
INFO:root:Converting journal trades to DataFrame


frozenset({'m15', 'h4', 'type_2_h4', 'type_1_d', 'type_1_m1', 'm5', 'd', 'type_2_d', 'M', 'type_2_M', 'type_1_m5', 'type_2_w', 'type_1_h1', 'm1', 'type_2_h1', 'type_1_m30', 'type_1_h4', 'type_1_m15', 'm30', 'w', 'h1', 'type_1_w', 'type_2_m5', 'type_1_M', 'type_2_m30', 'type_2_m15', 'type_2_m1'})


In [18]:
# Generate the feature dataframe
full_df

,type_1_m1,type_3_m15,entry_price,sl_price,tp_price,SL_distance,TP_distance,side,entry_time,risk_reward_ratio,...,return,htf_poi_ltf_confirmation,outcome,potential_risk_reward,potential_return,potential_price,has_type_1,has_type_2,has_type_3,has_time_frame
0,True,True,1.1000,1.0950,1.1100,0.0050,0.0100,long,2024-12-29 02:52:04.820735,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,False
1,False,False,1.2000,1.1850,1.2100,0.0150,0.0100,long,2024-12-29 02:52:04.820766,0.666667,...,-0.0145,True,loss,NaN,NaN,NaN,False,True,False,False
2,False,False,1.3000,1.2970,1.3300,0.0030,0.0300,long,2024-12-25 10:00:00.000000,10.000000,...,NaN,False,NaN,NaN,NaN,NaN,False,False,False,False
3,False,False,1.3000,1.2950,1.9100,0.0050,0.6100,long,2024-12-25 10:00:00.000000,10.000000,...,0.0050,True,win,5.000000,0.0150,1.3150,False,False,False,False
4,True,False,1.1378,1.1321,1.1663,0.0057,0.0285,long,2024-12-29 02:52:04.823948,5.000000,...,0.0117,False,win,5.245614,0.0299,1.1677,True,True,False,False
5,False,False,1.1331,1.1250,1.1810,0.0081,0.0479,long,2024-12-29 02:52:04.823976,5.913580,...,0.0180,False,win,1.518519,0.0123,1.1454,True,True,False,False
6,False,False,1.1161,1.1103,1.1595,0.0058,0.0434,long,2024-12-29 02:52:04.824029,7.482759,...,NaN,False,NaN,8.465517,0.0491,1.1652,True,True,False,False
7,False,False,1.1125,1.1046,1.1300,0.0079,0.0175,long,2024-12-29 02:52:04.824046,2.215190,...,NaN,True,NaN,2.658228,0.0210,1.1335,True,True,False,False
8,False,False,1.1495,1.1404,1.1857,0.0091,0.0362,long,2024-12-29 02:52:04.824063,3.978022,...,-0.0176,False,loss,2.549451,0.0232,1.1727,True,True,False,False
9,False,False,1.1439,1.1350,1.1559,0.0089,0.0120,long,2024-12-29 02:52:04.824080,1.348315,...,-0.0053,True,loss,1.438202,0.0128,1.1567,True,True,False,False


In [19]:
full_df.dtypes

type_1_m1                             bool
type_3_m15                            bool
entry_price                        float64
sl_price                           float64
tp_price                           float64
SL_distance                        float64
TP_distance                        float64
side                                object
entry_time                  datetime64[ns]
risk_reward_ratio                  float64
confidence_1                        object
confidence_2                        object
confidence_3                        object
confidence_4                        object
confidence_5                        object
numerical_confidence               float64
session                             object
initial_risk_reward                float64
initial_return                     float64
management_strategy                 object
type_1_m5                             bool
type_1_m15                            bool
type_1_m30                            bool
type_1_h1  

In [35]:
!rm -rf output

In [30]:
j.to_markdown(output_dir='output')


INFO:root:Converting trades to markdown
INFO:root:Writing index markdown


INFO:root:Calculating simple statistics
INFO:root:Analyzing tag relevance
INFO:root:Finding best single tags and subsets
INFO:root:Finding best single tags
INFO:root:Finding best tag subsets
INFO:root:Finding best tag subsets
INFO:root:Calculating tags statistics
INFO:root:Plotting statistics
